# Skill assessment: NWP combination
***

**Author**: Chus Casado Rodríguez<br>
**Date**: 09-02-2024<br>


**Introduction**:<br>
In this notebook I will analyse the EFAS skill in predicting flood events in general, i.e., looking whether events where predicted at some point in time, regardless of neither the offset nor the duration of the event.

In [1]:
import os
path_root = os.getcwd()
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
from matplotlib.ticker import MultipleLocator
import cartopy.crs as ccrs
import seaborn as sns
from datetime import datetime, timedelta
from tqdm import tqdm_notebook
import pickle
import yaml
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, KFold
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path

os.chdir('../py/')
from config import Config
from compute import *
from convert import dict2da
from optimize import find_best_criterion, find_best_criteria, find_best_criteria_cv
from plot.results import * 
from plot.maps import create_cmap, map_stations, map_hits, map_skill, map_events, gauge_legend
os.chdir(path_root)

In [2]:
# Set the default text font size
plt.rc('font', size=15)
# Set the axes title font size
plt.rc('axes', titlesize=16)
# Set the axes labels font size
plt.rc('axes', labelsize=15)
# Set the font size for x tick labels
plt.rc('xtick', labelsize=13)
# Set the font size for y tick labels
plt.rc('ytick', labelsize=13)
# Set the legend font size
plt.rc('legend', fontsize=13)
# Set the font size of the figure title
plt.rc('figure', titlesize=17)

# projection that will be used in the maps
proj = ccrs.LambertAzimuthalEqualArea(central_longitude=10,
                                      central_latitude=52, 
                                      false_easting=4321000, 
                                      false_northing=3210000,
                                      globe=ccrs.Globe(ellipse='GRS80'))

# extent of the maps: [min_lon, max_lon, min_lat, max_lat]
extent = [-10, 44, 28, 70]

## 1 Configuration

In [3]:
config_path = Path('../conf')
config = Config.load_from_yaml(config_path / 'config_COMB_leadtime_ranges.yml')

### 1.1 Reporting points

In [4]:
# area threshold
area_threshold = config.reporting_points['area']

# reporting points
path_stations = config.reporting_points['output']
file_stations = path_stations / f'reporting_points_over_{area_threshold}km2.parquet'

# catchments
catchments = config.reporting_points['catchments']

# rivers
rivers_shp = config.reporting_points['rivers']
if rivers_shp is not None:
    rivers = gpd.read_file(rivers_shp)
    
# minimum performance required from the reporting points
min_kge = config.reporting_points['KGE']

### 1.2 Hits

In [5]:
# return period
rp = config.discharge['return_period']['threshold']

# lead time ranges
leadtime = config.hits['leadtime']

# parameters of the rolling window used to compute hits
window = config.hits['window']

# dissagregate the analysis by seasons?
# seasonality = config.hits['seasonality']

# path that contains the NetCDFs with hit, misses and false alarms pro
path_in = config.hits['output']
if leadtime is None:
    time_agg = 'all_leadtimes'
elif len(leadtime) == 10:
    time_agg = 'daily'
elif len(leadtime) == 20:
    time_agg = '12h'
else:
    time_agg = '_'.join([str(lt + 12) for lt in leadtime])
path_in = path_in / f'{rp}/COMB/{time_agg}/window_{window}'

### 1.3 Skill

In [6]:
# current operationa criteria
current_criteria = config.skill['current_criteria']

# fixed notification criteria
min_leadtime = config.skill['leadtime']
min_area = config.skill['area']

# coefficient of the fbeta-score
beta = config.skill['beta']
metric = f'f{beta}'

# optimization parameters
optimization = config.skill['optimization']
kfold = optimization['kfold']
train_size = optimization['train_size']
stratify = optimization['stratify']
tolerance = optimization['tolerance']
min_spread = optimization['minimize_spread']
del optimization

# path where results will be saved
path_out_root = config.skill['output']
if min_kge is not None:
    kge = f'kge_{min_kge}'
else:
    kge = 'no_kge'
path_out_root = path_out_root / f'{rp}/COMB/{time_agg}/window_{window}/{kge}'
path_out = path_out_root / metric
path_out.mkdir(parents=True, exist_ok=True)

## 2 Data
### 2.1 Reporting points

In [ ]:
# load table of fixed reporting points
stations = pd.read_parquet(file_stations)
stations[['X', 'Y', 'area']] = stations[['X', 'Y', 'area']].astype(int)
stations.sort_values('n_events_obs', ascending=True, inplace=True)

# select stations that belong to the selected catchments
if catchments is not None:
    if isinstance(catchments, list) is False:
        catchments = [catchments]
    stations = stations.loc[stations.catchment.isin(catchments),:]

# remove points with a performance (KGE) lower than the established threshold
if min_kge is not None:
    mask_kge = ~(stations.KGE <= min_kge)
    stations = stations.loc[mask_kge]
else:
    # remove station with erroneous behaviour
    stations = stations.loc[~(stations.n_events_obs >= 6)]

In [ ]:
# mask stations with events
col_events = f'obs_events_{rp}'
stations_w_events = (stations[col_events] > 0)

print('All points')
print('----------')
print(f'no. reporting points:\t\t{stations.shape[0]}')
print('no. stations with events:\t{0}'.format(stations_w_events.sum()))
print('no. observed events:\t\t{0}'.format(stations[col_events].sum()))

# select stations according to catchment area
if min_area > area_threshold:
    stations_optimize = stations.loc[stations.area >= min_area].index
else:
    stations_optimize = stations.index

print('\nPoints selected for otimization')
print('-------------------------------')
print(f'no. reporting points:\t\t{len(stations_optimize)}')
print('no. stations with events:\t{0}'.format((stations.loc[stations_optimize, col_events] > 0).sum()))
print('no. observed events:\t\t{0}'.format(stations.loc[stations_optimize, col_events].sum()))

# suffix that will be used when saving plots
suffix = f'{min_area}km2_{len(stations_optimize)}points'

**Distribution of catchment area**

In [ ]:
xmin = area_threshold
xmax = 500001 #np.ceil(stations.area.max() / 500) * 500
bins = np.arange(xmin, xmax, 500).astype(int)

fig, ax = plt.subplots(figsize=(6, 5.5))
sns.histplot(stations.area, ax=ax, bins=bins, alpha=.5, label='all')
sns.histplot(stations[stations_w_events].area, ax=ax, alpha=.5, color='orange', bins=bins, label='w/ events')
ax.axvline(min_area, color='k', ls=':', lw=.75)
ax.set(xlabel='area (km²)', ylabel='no. reporting points', xlim=(xmin, 50000));
ax.xaxis.set_major_locator(MultipleLocator(10000))
ax.xaxis.set_minor_locator(MultipleLocator(2000))
ax.yaxis.set_major_locator(MultipleLocator(100))
ax.yaxis.set_minor_locator(MultipleLocator(20))
ax.spines[['right', 'top']].set_visible(False)
ax.legend(frameon=False);

plt.savefig(path_out_root / f'area_distribution_{suffix}.jpg', dpi=300, bbox_inches='tight')

> ***Figure 1**. Distribution of the reporting points according to catchment area. The complete set of reporting points is shown in blue, and the subset of reporting points with observed flood events during the study period is shown in red. The balck, dotted line represents the minimum catchment area that will be used for the optimization of the notification criteria.*

**Map of number of "observed" events**

In [1]:
map_events(x=stations.loc[stations_optimize].X, 
           y=stations.loc[stations_optimize].Y, 
           events=stations.loc[stations_optimize][col_events],
           rivers=rivers,
           yscale='log',
           size=4, 
           save=path_out_root / f'map_observed_events_{suffix}.jpg')

NameError: name 'map_events' is not defined

> ***Figure 2**. Number of observed flood events during the study period.*

The geographical distribution of events is not even. There is a higher proportion of stations with events in Central Europe, British Isles and the Mediterranean catchments than in Estearn and North-Eastern Europe. During the study period there were major events in the Rhine, Meuse and Ebro, which can be seen in the map.

The reporting points with more than 5 flood events during the study period were removed, since it is suspicious that the 5-year return period was exceeded so many times in only 2 years of study period.

### 2.2 Hits, misses and false alarms

In [ ]:
# import hits for each station
hits_stn = xr.open_mfdataset(f'{path_in}/*.nc', combine='nested', concat_dim='id')

# extract selected stations
stations = stations.loc[set(stations.index).intersection(hits_stn.id.data)]
hits_stn = hits_stn.sel(id=stations.index.to_list()).compute()

# convert to NaN lead times that can't be reached due to model limitations or persistence
hits_stn = limit_leadtime(hits_stn)

# subset of the 'hits' dataset with the stations selected for the optimization
stations_optimize = list(set(stations_optimize).intersection(hits_stn.id.data))
hits_opt = hits_stn.sel(id=stations_optimize).sum('id', skipna=False)

## 3 Analysis

In this section I will compute the skill of the EFAS predictions in different ways. In all the following sections I will work with three metrics: $recall$, $precision$ and the $f_{beta}$ score. The three metrics are based in the contingency table of hits ($TP$ for true positives), false alarms ($FP$ for false positives) and misses ($FN$ for false negatives).

$$recall = \frac{TP}{TP + FN}$$
$$precision = \frac{TP}{TP + FP}$$
$$f_{beta} = \frac{(1 + \beta^2) \cdot TP}{(1 + \beta^2) \cdot TP + \beta^2 \cdot FN + FP}$$

### 3.1 Skill computation

In [ ]:
# skill by station
skill_stn = hits2skill(hits_stn, beta=beta)

# skill dataset for optimizing criteria
skill_opt = hits2skill(hits_opt, beta=beta)

# export
skill_opt.to_netcdf(path_out / f'skill_{suffix}.nc')

**Benchmark: NWP**

In [ ]:
# load skill of NWP
path_NWP = Path(str(path_out).replace('COMB', 'NWP'))
skill_NWP = xr.open_dataset(path_NWP / f'skill_{suffix}.nc')

# load optimized criteria for NWP
with open(path_NWP / f'optimized_criteria_{suffix}.pkl', 'rb') as f:
    criteria_NWP = pickle.load(f)

### 3.2 Analyse overall performance

In [ ]:
# to simplify the plot, they will show only the following values of persistence and leadtime
persistences = skill_opt.persistence.data # ['1/1', '2/2', '3/3']

if min_leadtime is None:
    min_leadtime = int(skill_opt.leadtime.min().data)
if leadtime is None:
    leadtimes = [12, 36, 60, 84]
else:
    leadtimes = np.array(leadtime) + 12

**Skill vs lead time for several probability thresholds**

In [ ]:
# define colour map
top = cm.get_cmap('Oranges', 128)
bottom = cm.get_cmap('Blues_r', 128)
newcolors = np.vstack((top(np.linspace(0.2, .95, 128)),
                       bottom(np.linspace(.05, .8, 128))))
OrBu = ListedColormap(newcolors, name='OrangeBlue')

In [ ]:
At = np.unique(np.diff(skill_opt.leadtime))
if (len(At) == 1) & (At[0] == 24):
        
    xmax = (skill_opt.leadtime.max().data - 12) / 24
    
    # versus the best NWP: EUE
    benchmark_file = Path(str(path_out_root).replace('COMB', 'NWP')) / 'skill_EUE_benchmark_leadtime.csv'
    benchmark = xr.Dataset.from_dataframe(pd.read_csv(benchmark_file, index_col='leadtime'))
    benchmark = benchmark.set_coords(['model', 'probability', 'persistence'])
    for pers in ['1/1']:#persistences: # skill_opt.persistence.data:
        file = path_out / 'skill_probability_leadtime_{0}_COMB_vs_ENS.jpg'.format(path_out, metric, pers.replace('/', '-'))
        plot_skill_by_probability(skill_opt,
                                  probability=np.round(np.arange(.1, .96, .1), 3),
                                  metric='f0.8',
                                  persistence=pers,
                                  coldim='approach',
                                  benchmark=benchmark,
                                  label='ENS',
                                  lw=1,
                                  alpha=.5,
                                  ref_lt=60,
                                  ref_p=None, xlabel='lead time (d)', cmap=OrBu, xlim=(0, xmax),
                                  save=file)
        
    # versus the current criteria
    for pers in ['1/1']:#persistences: # skill_opt.persistence.data:
        file = path_out / 'skill_probability_leadtime_{0}_COMB.jpg'.format(path_out, metric, pers.replace('/', '-'))
        plot_skill_by_probability(skill_opt, probability=np.round(np.arange(.1, .96, .1), 3), metric='f0.8', persistence=pers,
                                  coldim='approach', benchmark=skill_opt.sel(current_criteria), l=1, alpha=.5, ref_lt=60,
                                  ref_p=None, xlabel='lead time (d)', cmap=OrBu, xlim=(0, xmax),
                                  save=file)

**Skill vs probability for several persistences**

In [ ]:
# define colour map
top = cm.get_cmap('Oranges', 128)
bottom = cm.get_cmap('Blues_r', 128)
newcolors = np.vstack((top(np.linspace(.8, 0.2, 128)),
                       bottom(np.linspace(.8, .2, 128))))
OrBu = ListedColormap(newcolors, name='OrangeBlue')

# versus the best NWP: EUE
path_NWP = Path(str(path_out).replace('COMB', 'NWP'))
benchmark_file = path_NWP / 'skill_EUE_benchmark_leadtime.csv'
benchmark_NWP = xr.Dataset.from_dataframe(pd.read_csv(benchmark_file, index_col='leadtime'))
benchmark_NWP = benchmark_NWP.set_coords(['model', 'leadtime', 'persistence', 'probability'])

for lt in leadtimes:
    file = path_out / 'skill_persistence_probability_{lt:03}h_COMB_vs_ENS.jpg'
    plot_skill_by_persistence(skill_opt.sel(leadtime=lt),
                              coldim='approach',
                              metric=metric,
                              benchmark=benchmark_NWP.sel(leadtime=lt),
                              cmap=OrBu,
                              label='ENS',
                              save=None)#file)

In [ ]:
# versus the current criteria
for lt in skill_opt.leadtime.data:
    benchmark = skill_opt.sel(current_criteria).sel(leadtime=lt)
    file = path_out / f'skill_persistence_probability_{lt:03}h_COMB.jpg'
    plot_skill_by_persistence(skill_opt.sel(leadtime=lt),
                              coldim='approach',
                              metric=metric,
                              benchmark=benchmark,
                              cmap=OrBu,
                              save=file)

#### 3.2.3 Optimize criteria

In this section we will derive a optimal set of notification criteria based on the target skill metric. To avoid overfitting, the sample of reporting points is first divided in a training and a test subset. This division is done in a stratified way after randomly shuflling the points, to keep the proportion of observed events in the subsets and avoid geographic biases, respectively.

To increase the robustness of the optimization, cross-validation is applied if the parameter `kfold` is set in the configuration file. In that case, `kfold` subsets of stations are generated, again in a stratified and random manner. The average skill over the subsets will be the data used to find the optimal criteria.

In [ ]:
seed = 2

# divide stations in train and test samples
X = stations_optimize
y = stations.loc[X, 'n_events_obs']
if stratify:
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size, random_state=seed, shuffle=True, stratify=y)
else:
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size, random_state=seed, shuffle=True)

best_criteria = {}
best_model = {}
for leadtime in hits_stn.leadtime.data:
    
    print(f'leadtime: {leadtime:>3} h')
    print('---------------\n')
    
    # subset of the 'hits' dataset for the training and test sets
    hits_train = hits_stn.sel(id=Xtrain, leadtime=leadtime)
    hits_test = hits_stn.sel(id=Xtest, leadtime=leadtime)

    # optimize the notification criteria
    if kfold is not None: # apply a cross-validation approach
        skill_train, best_criteria_lt = find_best_criteria_cv(hits_train,
                                                           stations.loc[Xtrain, 'n_events_obs'],
                                                           dims=list(min_spread),
                                                           kfold=kfold, train_size=train_size, random_state=seed, stratify=stratify,
                                                           beta=beta, tolerance=tolerance, min_spread=list(min_spread.values()))
    else:
        # skill of the training sample
        skill_train = hits2skill(hits_train.sum('id', skipna=True), beta=beta)
        # best criteria for each approach
        best_criteria_lt = find_best_criteria(skill_train, dims=list(min_spread),
                                           metric=metric, tolerance=tolerance, min_spread=list(min_spread.values()))

    # extract criteria as a dictionary
    dims = [var for var in best_criteria_lt if var not in ['recall', 'precision', metric]]
    best_criteria_lt = {app: {dim: best_criteria_lt.sel(approach=app)[dim].data for dim in dims} for app in best_criteria_lt.approach.data}
    for key in best_criteria_lt:
        best_criteria_lt[key]['approach'] = key
    best_criteria[leadtime] = best_criteria_lt
    
    # add current criteria
    best_criteria[leadtime]['current'] = current_criteria
    
    # skill of the train set for the optimized criteria
    skill_train = dict2da({key: skill_train.sel(sel) for key, sel in best_criteria_lt.items()}, dim='approach')

    # skill of the test set for the optimized criteria
    skill_test = hits2skill(hits_test.sum('id', skipna=True), beta=beta)
    skill_test = dict2da({key: skill_test.sel(sel) for key, sel in best_criteria_lt.items()}, dim='approach')

    # performance of the complete set of stations
    skill_all = dict2da({key: skill_opt.sel(leadtime=leadtime).sel(sel) for key, sel in best_criteria_lt.items()}, dim='approach')

    # plot results of the optimization
    plot_skill_training(skill_train,
                        skill_test,
                        skill_all,
                        ylim=(-0.05, 1.05),
                        xdim='approach',
                        save=path_out / f'skill_training_{suffix}_{leadtime:03}h.jpg')

    # print on screen results
    for key, dct in best_criteria_lt.items():
        print(key.replace('_', ' '))
        print('.' * len(key))
        for label, value in dct.items():
            if label == 'approach':
                continue
            print(f'{label}:\t{value}')
        print('{0}(train|test):\t{1:.3f}|{2:.3f}\n'.format(metric,
                                                         skill_train.sel(approach=key).median()[metric].data,
                                                         skill_test.sel(approach=key).median()[metric].data))

    # model with the highest skill
    best_model[leadtime] = str(skill_test[metric].idxmax('approach').data)

# export best criteria
file = path_out / f'optimized_criteria_{suffix}.pkl'
with open(file, 'wb') as f:
    pickle.dump(best_criteria, f)

> ***Figure 6**. Skill resulting of the otpimization process for each of the methods used to combine the meteorological forcings: 1D+1P, one deterministic and 1 probabilistic; MM, model mean; MW, member weighted; BW, Brier weihted; C, current operational criteria. If cross-validation was applied, the boxplots show the variance in the skill among the kfolds; if not, the black dots represent the skill of the training set. In either case, the orange dot represent the skill of the test set and the blue dots that of the complete set of reporting points.*

According to the performance on the test set (orange dots), **the optimal criteria is the one that uses the _member weighted_ approach with no persistence and a probability threshold of 40%**. However, for the training set the highest-performing approach was _1 deterministic and 1 probabilistic_, but this approach seems to suffer from overfitting, since it is the second lowest-performing in the test set.

Since the target metric benefits precision over recall, the precision values are higher than those of recall. This difference is enhanced for the _member weighted_ (the best-performing) and the _current_ criteria.

Persistence is not necessary in two out of four approaches: _1 deterministic and 1 probabilistic_, _member weighted_. The other two approaches have an optimized persistence of $4/4$.</font>

In general, the optimized criteria, regarless of the approach, outperform the current operational criteria. 

### 3.3 Analyse skill by reporting points

Once we have optimized the notification criteria for each approach, we will have a look to the distribution of the hits/misses/false alarms and the skill.

In [ ]:
# define colormap for the skill metrics
top = cm.get_cmap('Oranges_r', 128)
bottom = cm.get_cmap('Blues', 128)
newcolors = np.vstack((top(np.linspace(0.2, .95, 128)),
                       bottom(np.linspace(0.05, .8, 128))))
OrBu = ListedColormap(newcolors, name='OrangeBlue')
cmap_f1, norm_f1 = create_cmap(OrBu, np.arange(0, 1.01, 0.05), name='skill')

In [ ]:
# map hits/misses/false alarmas and performance for each of the total probability approaches
s = 2
alpha = 1
for leadtime in [min_leadtime]:#leadtimes:
    for key, criteria in best_criteria[leadtime].items():

        title = '{0}\nprobability = {1}\npersistence = {2}'.format(key.replace('_', ' '), criteria['probability'], criteria['persistence'])

        # extract TP, FN, FP for this approach
        cols = ['TP', 'FN', 'FP']
        stations[cols] = hits_stn.sel(leadtime=leadtime).sel(criteria).to_pandas()[cols]
        
        # plot maps of TP, FN, FP
        map_hits(stations.loc[stations_optimize], cols=cols, mask=stations_w_events, s=s, alpha=alpha,
                 title=title,
                 save=path_out / f'hits_maps_reporting_points_{suffix}_{key}_{leadtime:03}h.jpg')

        # compute metrics
        cols = ['recall', 'precision', metric]
        stations[cols] = pd.concat((compute_skill(stations.TP, stations.FN, stations.FP, beta=beta)), axis=1)
        # plot maps of performance
        map_skill(stations.loc[stations_optimize], cols=cols,
                  bins=50, cmap=cmap_f1, norm=norm_f1,
                  s=s, alpha=alpha,
                  title=title,
                  save=path_out / f'skill_maps_reporting_points_{suffix}_{key}_{leadtime:03}h.jpg')
        
stations.drop(['TP', 'FN', 'FP', 'recall', 'precision', metric], axis=1, inplace=True)

> ***Figure 7**. Maps of hits, misses and false alarms for the criteria otimized for each approach. The colour scale changes depending on the variable; orange (darker orange) means worse values, whereas blue (darker blue) better values. In the case of hits (TP) and misses (FN) a mask has been applied to remove reporting points with no observed events (gray points), since none of these variables can be computed if there are no observations to predict or miss. The histograms at the bottom show the distributions of hits, misses and false alarms over the whole domain.*

> ***Figure 8**. Maps of skill for the criteria otimized for each approach. Orange values represent poor skill, whereas blue values high skill; gray dots represent points for which the metric can not be computed. The histograms at the bottom show the distribution of skill over the whole domain.*

**Map of the skill metrics of the optimized vs the current criteria**

The following plot was designed for the article. It shows the results of the three skill metrics with an enfasis on the f-score, as it is the target metric. The difference between the skill metrics with the optimized and the current criteria are computed and shown in the plot through the direction of the arrows.

In [ ]:
lt = min_leadtime
key = 'brier_weighted' # 'current'

# skill of the benchmark, i.e., the current notification criteria
benchmark = hits_stn.sel(best_criteria[lt]['current']).sel(leadtime=lt).to_pandas()
R, P, F = compute_skill(benchmark.TP, benchmark.FN, benchmark.FP, beta=beta)
stations[['recall_bench', 'precision_bench', 'fscore_bench']] = pd.concat((R, P, F), axis=1)

# skill of the optimized notification criteria
opt = hits_stn.sel(best_criteria[lt][key]).sel(leadtime=lt).to_pandas()
R, P, F = compute_skill(opt.TP, opt.FN, opt.FP, beta=beta)
stations[['recall_opt', 'precision_opt', 'fscore_opt']] = pd.concat((R, P, F), axis=1)

# difference in skill between the optimized and the current criteria
for met in ['fscore', 'recall', 'precision']:
    stations[f'delta_{met}'] = stations[f'{met}_opt'] - stations[f'{met}_bench']

In [ ]:
# define colour map for the following plots
Bu = ListedColormap(plt.cm.get_cmap('Blues', 128)(np.linspace(.2, .9, 128)), 'blues')
cmap_f1, norm_f1 = create_cmap(Bu, np.arange(0, 1.01, 0.1), name='skill', specify_color=(0, 'chocolate'))

# select only stations used for the optimization
stns = stations.loc[stations_optimize].copy()

In [ ]:
s=12
scale = 60
alpha = .9
factor = 1 / np.cos(np.deg2rad(20)) # 20º corresponds to skill 1

# create figure
h, b = extent[1] - extent[0], extent[3] - extent[2]
width = 8
fig = plt.figure(figsize=(1.5 * width, width * b / h))

# plot f-score
met = 'fscore'
col = f'{met}_opt'
ax_main = plt.axes([0., 0., 0.605, 1.], projection=proj)
ax_main.set_extent(extent)
stns.sort_values(col, ascending=True, inplace=True)
map_stations(x=stns.X, 
             y=stns.Y,
             z=stns[col],
             theta=np.arccos(stns[f'delta_{met}'] / factor), # remove this line to plot dots instead of arrows
             mask=stns[col].isnull(),
             rivers=rivers,
             size=s, scale=scale, cmap=cmap_f1, norm=norm_f1, 
             #headaxislength=0,
             ax=ax_main)
ax_main.set_title(r'$f_{0.8}$', fontsize=18)

# plot precision and recall
for i, met in enumerate(['precision', 'recall']):
    col = f'{met}_opt'
    ax = plt.axes([0.6, 0.525 * i, .34, .475], projection=proj)
    ax.set_extent(extent)
    stns.sort_values(col, ascending=True, inplace=True)
    map_stations(x=stns.X, 
                 y=stns.Y, 
                 z=stns[col], 
                 theta=np.arccos(stns[f'delta_{met}'] / factor), # remove this line to plot dots instead of arrows
                 size=s/4, scale=42, width=.002, 
                 #headaxislength=0,
                 cmap=cmap_f1, norm=norm_f1, 
                 ax=ax)
    ax.set_title(met, fontsize=18)

# Colorbar
cax = fig.add_axes([.2, -.1, .333, .01666])
cbar = fig.colorbar(map_stations.colorbar, cax=cax, orientation='horizontal')
cbar.set_label('skill (-)')
cbar.ax.tick_params(size=0)
ticks = [0, .2, .4, .6, .8, 1.]
cbar.set_ticks(ticks)
cbar.set_ticklabels(ticks, fontsize=15)
cbar.outline.set_edgecolor('none');

# gauge chart
cax2 = fig.add_axes([.6, -.2, .1, .2])
gauge_legend(scale=2.3, width=.015, title=f'$\Delta$ skill (-)', ax=cax2)


# export
filename = path_out / f'skill_maps_reporting_points_{suffix}_{key}_{leadtime:03}h.jpg'
#plt.savefig(filename, dpi=300, bbox_inches='tight')
print(f'The plot was saved in {filename}')

In [ ]:
# export station including skill
stations.drop(['n_events_obs', 'delta_fscore', 'delta_recall', 'delta_precision'], axis=1, inplace=True, errors='ignore')
stations.to_parquet(path_out / str(file_stations).split('\\')[-1])

**Roebber diagram**

In [ ]:
# extract best performance of each NWP and approach for `min_leadtime`
results = {'NWP': {},
          'COMB': {}}
for model, criteria in criteria_NWP[min_leadtime].items():
    results['NWP'][model] = skill_NWP.sel(criteria).sel(leadtime=min_leadtime).to_pandas()[['precision', 'recall']].values

for app, criteria in best_criteria[min_leadtime].items():
    if len(app.split('_')) > 1:
        app = ''.join([x[0] for x in app.split('_')]).upper()
    results['COMB'][app] = skill_opt.sel(criteria).sel(leadtime=min_leadtime).to_pandas()[['precision', 'recall']].values

# define colour map
top = cm.get_cmap('Blues_r', 128)
bottom = cm.get_cmap('Oranges', 128)
newcolors = np.vstack((top(np.linspace(0.2, .95, 128)),
                       bottom(np.linspace(.05, .8, 128))))
BuOr = ListedColormap(newcolors, name='BlueOrange')
colors_models = ListedColormap(BuOr(np.linspace(0, 1, 4))).colors

fig, ax = roebber_diagram(metric='fscore', beta=beta, lw=.25)

# benchmark
ax.scatter(*results['COMB']['current'], marker='x', c='k', label='current', zorder=10)
handles1, labels1 = ax.get_legend_handles_labels()

# NWP
labels = {'EUE': 'ENS', 'COS': 'COS', 'EUD': 'HRES', 'DWD': 'DWD'}
for key, color in zip(['EUE', 'COS', 'EUD', 'DWD'], colors_models):
    ax.scatter(*results['NWP'][key], marker='^', facecolors='none', edgecolors=color, label=labels[key], zorder=9)
handles2, labels2 = ax.get_legend_handles_labels()
# ax.scatter(-1, -1, color='w', label=' ')

# COM
for key, color in zip(['BW', 'MW', 'MM', '1D+1P'], colors_models):
    ax.scatter(*results['COMB'][key], marker='o', facecolors='none', edgecolors=color, label=key, zorder=9)
handles3, labels3 = ax.get_legend_handles_labels()

len2 = len(handles2)
handles3, labels3 = handles3[len2:], labels3[len2:]
len1 = len(handles1)
handles2, labels2 = handles2[len1:], labels2[len1:]

fig.legend(handles1, labels1, frameon=False, bbox_to_anchor=[1., .0, .15, .84], loc=2)
fig.text(1.05, .72, 'NWP', ha='left', va='top', fontsize=13)
fig.legend(handles2, labels2, frameon=False, bbox_to_anchor=[1., .0, .15, .7], loc=2)
fig.text(1.05, .42, 'COMB', ha='left', va='top', fontsize=13)
fig.legend(handles3, labels3, frameon=False, bbox_to_anchor=[1, .0, .15, .4], loc=2);

plt.savefig(path_out / f'roebber_{suffix}_{min_leadtime}h.jpg',
            dpi=300, bbox_inches='tight');

### 3.4 Analyse skill by catchment area

So far we have analyzed only stations with a catchment area larger or equal than a fixed value (2000 km²). Also, in the optimization of the notification criteria this minimum catchment area was fixed.

In this section we will analyze how results change according to the catchment area. First, we will see the evolution of skill over catchment area for the notification criteria optimized for a minimum catchment area. Later, we will derive a new optimization criteria in which the probability threshold varies according to catchment area. This derivation is repeated for every approach, and the persistence criterion is fixed for each approach to the value optimized in the previous sections.

In [ ]:
# define an array of catchment area thresholds
area_max = np.ceil(stations.area.max() / 500) * 500
areas = define_area_ranges(500, area_max, scale='semilog')

# no. stations and events by catchment area threshold
stations_area = summarize_by_area(stations.area, stations[col_events], areas)

# hits and skill by catchment area
areas = stations_area[stations_area.n_events_obs > 0].index.to_list()
hits_area = hits_by_area(hits_stn.sel(leadtime=leadtimes), stations.area, areas)
skill_area = hits2skill(hits_area, beta=beta)
skill_area = skill_area.dropna('area', how='all')

In [ ]:
criteria_area = {}
for area in tqdm_notebook(areas):
        
    # divide stations in train and test samples
    X = stations.loc[stations.area >= area].index
    if len(X) == 0:
        break
    y = stations.loc[X, col_events]
    if stratify:
        Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size, random_state=seed, shuffle=True, stratify=y)
    else:
        Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size, random_state=seed, shuffle=True)

    # subset of the 'hits' dataset for the training and test sets
    hits_train = hits_stn.sel(id=Xtrain, leadtime=leadtimes)

    # optimize the notification criteria
    try:
        if kfold is not None: # apply a cross-validation approach
            skill_train, criteria = find_best_criteria_cv(hits_train,
                                                          stations.loc[Xtrain, col_events],
                                                          dims='probability',
                                                          kfold=kfold, train_size=train_size, random_state=seed, stratify=stratify,
                                                          beta=beta, tolerance=tolerance, min_spread=min_spread['probability'])
        else:
            # skill of the training sample
            skill_train = hits2skill(hits_train.sum('id', skipna=True), beta=beta)
            # best criteria for each approach
            criteria = find_best_criteria(skill_train, dims='probability',
                                          metric=metric, tolerance=tolerance, min_spread=min_spread['probability'])

        criteria_area[area] = criteria['probability']
            
    except:
        continue
        
criteria_area = dict2da(criteria_area, dim='area')

#### 3.4.1 Stations and events according to catchment area

In [ ]:
lw = 1
alpha = 1.
c1 = cm.get_cmap('Oranges', 128)(.66)
c2 = cm.get_cmap('Blues', 128)(.66)

fig, ax = plt.subplots(figsize=(6, 6))

ax.plot(stations_area.index, stations_area.n_stations, alpha=alpha, c=c1, lw=lw, label='points', zorder=2)
ax.plot(stations_area.index, stations_area.n_events_obs, alpha=alpha, c=c2, lw=lw, label='events', zorder=1)
ax.axvline(x=min_area, ls=':', lw=.5, color='k', zorder=0)
ax.text(2000, 0, 'current limit', va='bottom', ha='right', rotation=90)#, fontsize=12)
ymin, ymax = 0, np.ceil(stations_area.max().max() / 1000) * 1000
yticks = np.arange(ymin, ymax * 1.02, 500).astype(int)
ax.set(xlabel='area ≥ (km²)',
       xscale='log',
       xlim=(500, area_max),
       ylim=(ymin - ymax * .02, ymax * 1.02),
       ylabel='count (-)',
       yticks=yticks)
ax.spines[['right', 'top']].set_visible(False)
ax.legend(frameon=False, loc=1);

plt.savefig(path_out_root / f'points_observedEvents_vs_area_{suffix}.jpg', dpi=300, bbox_inches='tight');

> ***Figure 9**. Number of reporting points (orange) and observed events (blue) by catchment area.*

The plot represents both the number of reporting points and the number of observed events over a increasing catchment area threshold. Note that the X axis is in logarithmic scale, and that the primary Y axis (reporting points) has a scale double than the secondary Y axis (events).

There is a clear 2:1 relation between the number of reporting points and the number of observed events. Both variables dicrease exponentially with catchment area. However, for small catchments (lower than 2000 km² approx.) the 2:1 relation disappears; the number of events increases faster than that of reporting points with dicreasing area

#### 3.4.2 Current vs optimized criteria

**Hits, misses and false alarms**

In [ ]:
for lt in [min_leadtime]:
    plot_hits_by_variable(hits_area.sel(leadtime=lt),
                          optimal_criteria=best_criteria[lt],
                          variable='area',
                          coldim='approach',
                          reference=min_area,
                          current_criteria=current_criteria,
                          xscale='log',
                          xlabel='area ≥ (km²)',
                          xlim=(criteria_area.area.min(),
                                criteria_area.area.max()),
                          loc_legend=[.87, .8, .2, .1],
                          save=None)#f'{path_out}{metric}/hits_vs_area_{suffix}_{lt:03}h.jpg')

> ***Figure 10**. Evolution of hits, misses and false alarms with catchment area threshold. Each plot represents a different approach to combine the meteorological forcings. The primary Y axis is normalized by the number of observed events to allow for comparison; the secondary Y axis indicates the probability threshold. The continuous lines are the hits, whereas the shadows are the false alarms; the difference between the reference line ($\frac{x}{obs}=1$) and the hits are the misses. The dotted lines are the probability thresholds. Black objects represent the current operational criteria and blue ones the criteria optimized for a fixed area threshold (represented by a vertical, solid, black line).*

**Skill**

In [ ]:
for lt in [min_leadtime]:#leadtimes:
    plot_skill_by_variable(skill_area.sel(leadtime=lt), 
                           optimal_criteria=best_criteria[lt], 
                           variable='area', 
                           coldim='approach',
                           reference=min_area, 
                           metric=metric, 
                           current_criteria=current_criteria, 
                           shades=False,
                           xscale='log',
                           xlabel='area ≥ (km²)',
                           xlim=(skill_area.area.min(), 3e5), 
                           loc_text=3, 
                           loc_legend=[.87, .8, .2, .1],
                           save=None)#f'{path_out}{metric}/skill_vs_area_{suffix}_{lt:03}h.jpg')

In [ ]:
colors = {'brier_weighted': colors_models[0],
          'member_weighted': colors_models[1],
          'model_mean': colors_models[2],
          '1_deterministic_+_1_probabilistic': colors_models[3]}

for lt in [min_leadtime]:#leadtimes:
    plot_skill_by_area(skill_area.sel(leadtime=lt),
                       optimal_criteria=best_criteria[lt],
                       current_criteria=current_criteria,
                       reference = min_area,
                       xscale='log',
                       xlabel='area ≥ (km²)',
                       xlim=(skill_area.area.data.min(), 3e5),
                       metric=metric,
                       colors=colors,
                       # plot_prob=True,
                       save=path_out / f'skill_vs_area_{suffix}_{lt:03}h.jpg')

> ***Figure 11**. Evolution of skill with catchment area threshold. Each plot represents a different approach in which the meteorological forcings are combined. The primary Y axis indicates skill, and the secondary Y axis indicates the probability threshold. The continuous lines are the target skill score ($f_{0.8}$), and the shadows represent the difference between $precision$ and $recall$. The dotted lines are the probability thresholds. Black objects represent the current operational criteria and blue ones the criteria optimized for a fixed area threshold (represented by the vertical, solid, black line).*

#### 3.4.3 Fixed criteria vs area optimized criteria

In the previous section we have analyzed how the skill of the system varies over catchment area for a fixed value of the probability threshold. But, what if we tune the probability threshold according to catchment area? Would it improved the skill of the system?

**Hits, misses and false alarms**

In [ ]:
for lt in [min_leadtime]:#leadtimes:
    plot_hits_by_variable(hits_area.sel(leadtime=lt),
                          optimal_criteria=best_criteria[lt],
                          variable='area',
                          coldim='approach',
                          reference=min_area,
                          optimized_criteria=criteria_area.sel(leadtime=lt),
                          xscale='log', xlabel='area ≥ (km²)', xlim=(criteria_area.area.min(), criteria_area.area.max()),
                          loc_text=1, loc_legend=[.9, .8, .2, .1],
                          save=None)#path_out / f'hits_vs_area_varying_probability_{suffix}_{lt:03}h.jpg', )

> ***Figure 12**. Evolution of hits, misses and false alarms with catchment area threshold. Each plot represents a different approach to combine the meteorological forcings. The primary Y axis is normalized by the number of observed events to allow for comparison; the secondary Y axis indicates the probability threshold. The continuous lines are the hits, the shadows are the false alarms, and the difference between the reference line ($\frac{x}{obs}=1$) and the hits are the misses. The dotted lines show the probability threshold. Blue objects are the results for the criteria optimized for a fixed area threshold (represented by a vertical, solid, black line), and orange objects those for the criteria optimized for every area threshold.*

**Skill**

In [ ]:
for lt in [min_leadtime]:#leadtimes:
    plot_skill_by_variable(skill_area.sel(leadtime=lt),
                           optimal_criteria=best_criteria[lt],
                           variable='area', coldim='approach',
                           reference=min_area, metric=metric,
                           optimized_criteria=criteria_area.sel(leadtime=lt),
                           shades=False,
                           xscale='log', xlabel='area ≥ (km²)', xlim=(skill_area.area.data.min(), 3e5), loc_text=3, loc_legend=[.9, .8, .15, .1],
                           save=path_out / f'skill_vs_area_varying_probability_{suffix}_{lt:03}h.jpg')

> ***Figure 13**. Evolution of skill with catchment area threshold. Each plot represents a different approach in which the meteorological forcings are combined. The primary Y axis indicates skill, and the secondary Y axis indicates the probability threshold. The continuous lines are the target skill score ($f_{0.8}$), and the shadows represent the difference between $precision$ and $recall$. The dotted lines show the probability threshold. Blue objects are the results for the criteria optimized for a fixed area threshold (represented by a vertical, solid, black line), and orange objects those for the criteria optimized for every area threshold.*

***

**Heatmap for the layout figure in the paper**

In [ ]:
import matplotlib.patches as patches

cmap_f, norm_f = create_cmap('Blues', bounds=np.arange(.2, .71, .05), name='fscore')

da = skill_opt[metric].sel(approach='member_weighted', leadtime=60, probability=skill_opt.probability[::2])

fig, ax = plt.subplots(figsize=(7.6, 2))

plot_DataArray(da.sel(persistence=slice(None, None, -1)),
               cmap=OrBu, alpha=.75,
               xlabel='probability',
               xtick_step=2,
               ylabel='persistence',
               cbar_kws={'shrink': .8, 'label': r'$f_{0.8}$ (-)'},
               ax=ax)
xticks = np.arange(1, len(da.probability), 2)
xticklabels = da.isel(probability=xticks).probability.data
ax.set(xticks=xticks + .5,
       xticklabels=xticklabels)
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')
rect = patches.Rectangle((9, 5), 1, 1, facecolor='none', edgecolor='k', lw=1.5)
ax.add_patch(rect);

# plt.savefig('../docs/paper/Figures/paper_fscore_matrix.jpg', dpi=300, bbox_inches='tight');